# Probability grids

This notebook creates the time-dependent probability maps and writes them to file (`.nc` format), using the models trained in the previous notebook (`01-create_classifiers.ipynb`). This notebook must have been run previously.

## Notebook options

These cells set some of the important variables and definitions used throughout the notebook.

In [1]:
config_file = "notebook_parameters_deformation.yml"

In [2]:
from lib.load_params import get_params

params = get_params(config_file, notebook="02")

# If True, use training data extracted in notebook 00c
# Else, use pre-prepared training data downloaded from Zenodo
use_extracted_data = params["use_extracted_data"]

# Number of processes to use
n_jobs = params["n_jobs"]

# Overwrite any existing output files
overwrite = params["overwrite_output"]

# Control verbosity level of logging output
verbose = params["verbose"]

# Output directory
output_dir = params["output_dir"]

## Notebook setup

Imports, definitions, etc.

### Imports

In [3]:
import os
import time
from datetime import timedelta

import pandas as pd
from joblib import load

from lib.check_files import check_prepared_data
from lib.pu import create_probability_grids

### Input and output files

In [4]:
if use_extracted_data:
    data_dir = params["extracted_data_dir"]
else:
    data_dir = check_prepared_data("prepared_data", verbose=True)

columns = [
    "lon",
    "lat",
    "present_lon",
    "present_lat",
    "age (Ma)",
    "erosion (m)",
    "region",
]

data_filename = os.path.join(data_dir, "grid_data_deformation.csv")
training_filename = os.path.join(data_dir, "training_data_global_deformation.csv")

point_data = pd.read_csv(data_filename)
df_out = point_data[columns]
regions = list(pd.read_csv(training_filename)["region"].unique())

output_dir = os.path.join(output_dir, "global")

## Calculate probabilities

Deposit probability will be calculated for the gridded data and written to a CSV file.

In [5]:
for algorithm in ("PU", "SVM"):
    t0 = time.time()

    subdir = os.path.join(output_dir, algorithm)
    model_filename = os.path.join(subdir, f"classifier.joblib")
    probabilities_filename = os.path.join(
        subdir,
        "grid_probabilities.csv",
    )
    model = load(model_filename)

    point_x = point_data[model.feature_names_in_]
    p = model.predict_proba(point_x)[:, 1].ravel()
    probabilities = df_out.copy()
    probabilities["probability"] = p
    del p
    probabilities.to_csv(probabilities_filename, index=False)
    del probabilities, model
    duration = timedelta(seconds=time.time() - t0)
    if verbose:
        print(
            f"Calculating probabilities for {algorithm} model complete",
            f"(region: global; duration: {duration})",
        )

    for region in regions:
        region_code = "_".join(region.lower().split())
        model_filename = os.path.join(
            subdir,
            f"classifier_{region_code}.joblib",
        )
        if not os.path.isfile(model_filename):
            continue
        t0 = time.time()

        probabilities_filename = os.path.join(
            subdir,
            f"grid_probabilities_{region_code}.csv",
        )
        model = load(model_filename)

        try:
            model[-1].set_params(n_jobs=1)
        except ValueError:
            pass

        point_x = point_data[model.feature_names_in_]
        p = model.predict_proba(point_x)[:, 1].ravel()
        
        
        
        
        
        
        
        ()
        probabilities = df_out.copy()
        probabilities["probability"] = p
        del p
        probabilities.to_csv(probabilities_filename, index=False)
        del probabilities, model
        duration = timedelta(seconds=time.time() - t0)
        if verbose:
            print(
                f"Calculating probabilities for {algorithm} model complete",
                f"(region: {region}; duration: {duration})",
            )

del point_data

## Create probability maps

The probabilities calculated in the previous section will now be written to one netCDF file per time step:

In [6]:
for algorithm in ("PU", "SVM"):
    t0 = time.time()

    subdir = os.path.join(output_dir, algorithm)
    probabilities_filename = os.path.join(
        subdir,
        "grid_probabilities.csv",
    )
    grid_output_dir = os.path.join(
        subdir,
        "probability_grids",
    )
    os.makedirs(grid_output_dir, exist_ok=True)

    create_probability_grids(
        data=probabilities_filename,
        output_dir=grid_output_dir,
        threads=n_jobs,
        extent=(-180, 180, -90, 90),
    )
    duration = timedelta(seconds=time.time() - t0)
    if verbose:
        print(
            f"Creating grids for {algorithm} model complete",
            f"(region: global; duration: {duration})",
        )

    for region in regions:
        region_code = "_".join(region.lower().split())
        subdir = os.path.join(output_dir, algorithm)
        probabilities_filename = os.path.join(
            subdir,
            f"grid_probabilities_{region_code}.csv",
        )
        if not os.path.isfile(probabilities_filename):
            continue
        t0 = time.time()

        grid_output_dir = os.path.join(
            subdir,
            f"probability_grids_{region_code}",
        )
        os.makedirs(grid_output_dir, exist_ok=True)

        create_probability_grids(
            data=probabilities_filename,
            output_dir=grid_output_dir,
            threads=n_jobs,
            extent=(-180, 180, -90, 90),
        )
        duration = timedelta(seconds=time.time() - t0)
        if verbose:
            print(
                f"Creating grids for {algorithm} model complete",
                f"(region: {region}; duration: {duration})",
            )